In [45]:
import pandas as pd
import fasttext
import string
import time
import re

In [133]:
df_train = pd.read_csv("train_data.csv")
df_test = pd.read_csv("test_data.csv")

In [134]:
#标点符号：
#remove = str.maketrans('','',string.punctuation)
def token_labeled(x,y):#字符清洗
    label = '__label__'+str(y)
    remove = str.maketrans('','',string.punctuation)
    x = x.lower()
    x = re.sub('[\d]','',x)
    x = x.translate(remove)
    x = x.replace('\n','')
    x = label+' '+x
    return x
def token_unlabeled(x):
    remove = str.maketrans('','',string.punctuation)
    x = x.lower()
    x = re.sub('[\d]','',x)
    x = x.translate(remove)
    x = x.replace('\n','')
    return x

In [135]:
start = time.time()
df_train['cleaned'] = df_train.apply(lambda x:token_labeled(x.comment_text,x.toxic),axis = 1)
duration = time.time()-start
duration

11.835739612579346

In [136]:
train = list(df_train['cleaned'])
with open('train.train','w',encoding = 'utf-8') as f:
    f.write('\n'.join(train))

In [138]:
start = time.time()
model = fasttext.train_supervised('train.train',epoch =100,wordNgrams = 2)
duration = time.time()-start
model.save_model('fasttest.bin')

In [ ]:
#model = fasttext.load_model('fasttext.bin')

In [139]:
df_test['pre'] = df_test['comment_text'].apply(lambda x:token_unlabeled(x))
df_test

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,pre
0,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0,cocksucker before you piss around on my work
1,0005c987bdfc9d4b,Hey... what is it..\n@ | talk .\nWhat is it......,1,0,0,0,0,0,hey what is it talk what is it an exclusive g...
2,0007e25b2121310b,"Bye! \n\nDon't look, come or think of comming ...",1,0,0,0,0,0,bye dont look come or think of comming back to...
3,001810bf8c45bf5f,You are gay or antisemmitian? \n\nArchangel WH...,1,0,1,0,1,1,you are gay or antisemmitian archangel white t...
4,00190820581d90ce,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,0,1,0,1,0,fuck your filthy mother in the ass dry
...,...,...,...,...,...,...,...,...,...
10168,1688a31cc10fcd6d,| decline = I am declining your unblock reques...,0,0,0,0,0,0,decline i am declining your unblock request ...
10169,168911f6869a2038,Thank you - I'm glad I could help you. As alwa...,0,0,0,0,0,0,thank you im glad i could help you as always ...
10170,168915cdfe9be1ef,"""In point of fact the plot summary unaccountab...",0,0,0,0,0,0,in point of fact the plot summary unaccountabl...
10171,1689c37fdfa401cd,#NAME?,0,0,0,0,0,0,name


In [140]:
y_te = list(df_test['toxic'])
x_te = list(df_test['pre'])

In [169]:
def FTpre(x_te,y_te,model):
    n = len(x_te)
    n00 = 0
    n01 = 0
    n10 = 0
    n11 = 0
    list10 = list()
    start = time.time()
    for idx in range(n):
        result = model.predict(x_te[idx])[0][0]
        if result == '__label__1':
            if y_te[idx] == 1:
                n11 = n11+1
            else:
                n01 = n01+1
        else:
            if y_te[idx] ==1:
                n10 = n10+1
                list10.append(x_te[idx])
            else:
                n00 = n00+1

        if idx<10:
            print('\r'+'■'*((idx*30)//n)+str(round(idx*100/n,2))+'%',end = '')
        elif idx == 10:
            timecost = time.time()-start
            eta = (n-idx)/10*timecost
            print('\r'+'■'*((idx*30)//n)+str(round(idx*100/n,2))+'%'+' eta:'+str(round(eta,1))+'s',end = '')           
        else :
            eta = (n-idx)/10*timecost
            print('\r'+'■'*((idx*30)//n)+str(round(idx*100/n,2))+'%'+' eta:'+str(round(eta,1))+'s',end = '')
    print('\r'+'■'*30+'100%'+'completed!')

    return n00,n01,n10,n11,list10

In [170]:
n00,n01,n10,n11,list10 = FTpre(x_te,y_te,model)

■■■10.65% eta:919947.2s

■■■■■■20.5% eta:818540.5s

■■■■■■■■29.65% eta:724319.3s

■■■■■■■■■■■37.72% eta:641230.6s

■■■■■■■■■■■■■■46.74% eta:548325.0s

...

■■■■■■■■■■■■■■■■■■■■66.73% eta:342576.6s

■■■■■■■■■■■■■■■■■■■■■■76.43% eta:242688.0s

■■■■■■■■■■■■■■■■■■■■■■■■■85.99% eta:144216.2s

■■■■■■■■■■■■■■■■■■■■■■■■■■■■95.84% eta:42809.4s

■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■100%completed!


In [157]:
n = n00+n01+n10+n11
print(f'accuracy:{(n00+n11)/n}')
print(f'presicion:{n11/(n01+n11)}')
print(f'recall:{n11/(n10+n11)}')

accuracy:0.8999311904059766
presicion:0.9541892706449668
recall:0.6269306930693069


In [171]:
list10

['hey what is it  talk what is it an exclusive group of some wp talibanswho are good at destroying selfappointed purist who gang up any one who asks them questions abt their antisocial and destructive noncontribution at wpask sityush to clean up his behavior than issue me nonsensical warnings',
 'bye dont look come or think of comming back tosser',
 'im sorry im sorry i screwed around with someones talk page  it was very bad to do  i know how having the templates on their talk page helps you assert your dominance over them  i know i should bow down to the almighty administrators  but then again im going to go play outsidewith your mom   ',
 'name',
 'why cant you believe how fat artie is did you see him on his recent appearence on the tonight show with jay leno he looks absolutely awful if i had to put money on it id say that artie lange is a cant miss candidate for the  dead pool     kindly keep your malicious fingers off of my above comment  everytime you remove it i will repost it',

In [168]:
x_te[idx]

NameError: name 'idx' is not defined

In [152]:
n11

1583

In [160]:
n10

942

In [154]:
n01

76